In [ ]:
# Importing necessary libraries
import os      # Library for interacting with the operating system
import time    # Library for tracking the time taken to perform a task
import torch   # PyTorch library for building and training deep learning models
import torchvision   # A PyTorch library containing popular datasets, model architectures, and image transformations
import torchvision.transforms as transforms   # A module containing common image transformations
import torch.nn as nn   # PyTorch library containing various neural network layers and functions
from torch.nn import init   # A function for initializing the weights of a neural network
from torch.optim import lr_scheduler   # A PyTorch module for implementing various learning rate scheduling strategies
import functools   # A module providing various functions for higher-order programming
import numpy as np   # A library for numerical computations
import cv2 as cv   # OpenCV library for image and video processing
from PIL import Image   # A library for image processing and manipulation

In [ ]:
def make_dataset(directory):
    
    images = []  # create an empty list to store the image paths
    
    # check if the given directory is valid or not
    assert os.path.isdir(directory), '%s is not a valid directory' % directory

    # walk through the directory and subdirectories to find all image files
    for root, _, fnames in sorted(os.walk(directory)):
        for fname in fnames:
            path = os.path.join(root, fname)  # get the full path of the image file
            images.append(path)  # add the image path to the list of images
    
    return images  # return the list of image paths

In [ ]:
def __print_size_warning(ow, oh, w, h):
    
    if not hasattr(__print_size_warning, 'has_printed'):
        
        print("The image size needs to be a multiple of 4. "
              "The loaded image size was (%d, %d), so it was adjusted to "
              "(%d, %d). This adjustment will be done to all images "
              "whose sizes are not multiples of 4" % (ow, oh, w, h))
        
        __print_size_warning.has_printed = True

In [ ]:
def __make_power_2(img, base, method=Image.BICUBIC):
    
    """
    Resizes the input image to the nearest size that is a power of 2. If the image size is already a power of 2, then the
    original image is returned. Uses the PIL library for image resizing.

    Args:
        img (PIL.Image): Input image to resize.
        base (int): Base size to round the image dimensions to.
        method (int): Resampling method for image resizing. Default is Image.BICUBIC.

    Returns:
        (PIL.Image): Resized image.
    """
    
    ow, oh = img.size # Get the original width and height of the image.
    h = int(round(oh / base) * base) # Calculate the new height by rounding the original height to the nearest multiple of the base size.
    w = int(round(ow / base) * base) # Calculate the new width by rounding the original width to the nearest multiple of the base size.
    
    # If the new height and width are the same as the original height and width, return the original image.
    if h == oh and w == ow:
        return img

    # Otherwise, print a warning about resizing the image.
    __print_size_warning(ow, oh, w, h)
    
    # Resize the image to the new dimensions using the specified resampling method.
    return img.resize((w, h), method)

In [ ]:
def get_transform(method):
    
    """
    The get_transform function takes in a method argument and returns a PyTorch transform pipeline.

    The transform pipeline consists of three transforms:

    1. A custom lambda transform that resizes the image to the nearest multiple of 4 using the __make_power_2 function.
    2. The ToTensor transform, which converts the image to a PyTorch tensor.
    3. The Normalize transform, which normalizes the tensor values to have a mean of 0.5 and a standard deviation of 0.5 for each channel.
    
    Finally, the Compose transform is used to combine the individual transforms into a single pipeline.
    
    """
    
    # Create a list to hold all the transforms
    transform_list = []
    
    # Add a custom lambda transform that resizes the image to the nearest multiple of 4 using the __make_power_2 function
    transform_list.append(transforms.Lambda(lambda img: __make_power_2(img, base=4, method=method)))

    # Convert the image to a tensor
    transform_list += [transforms.ToTensor()]
    
    # Normalize the image tensor
    transform_list += [transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    
    # Combine the transforms into a single pipeline using the transforms.Compose function and return it
    return transforms.Compose(transform_list)

In [ ]:
def init_net(net, gpu_ids, init_type='normal', init_gain=0.02):

    # Check if any GPU is available and set the network to run on it
    if gpu_ids[0] != -1:
        assert(torch.cuda.is_available())
        net.to(gpu_ids[0])
        net = torch.nn.DataParallel(net, gpu_ids)  # multi-GPUs

    # Function to initialize weights and biases of the network
    def init_func(m): 
        
        classname = m.__class__.__name__
        
        # For Conv and Linear layers, initialize weight with normal distribution and bias with constant 0
        if hasattr(m, 'weight') and (classname.find('Conv') != -1 or classname.find('Linear') != -1):
            init.normal_(m.weight.data, 0.0, init_gain)
            if hasattr(m, 'bias') and m.bias is not None:
                init.constant_(m.bias.data, 0.0)
                
        # For BatchNorm2d layers, initialize weight with normal distribution and bias with constant 0
        elif classname.find('BatchNorm2d') != -1: 
            init.normal_(m.weight.data, 1.0, init_gain)
            init.constant_(m.bias.data, 0.0)

    # Apply the initialized function to the network
    net.apply(init_func)
    
    return net

In [ ]:
class AlignedDataset(torch.utils.data.Dataset):
    """
    A dataset class that reads aligned image pairs from the disk, where images in each pair have the same filename.
    """

    def __init__(self, dataroot, phase):
        """
        Args:
            dataroot (str): Root directory of the dataset.
            phase (str): 'train' or 'test' phase.
        """
        self.dataroot = dataroot
        self.phase = phase
        self.dir_AB = os.path.join(dataroot, phase)  # get the directory of the images
        self.AB_paths = sorted(make_dataset(self.dir_AB))  # get the paths of the images

    def __getitem__(self, index):
        """
        Args:
            index (int): Index of the pair of images.
        Returns:
            A dictionary containing the processed images and their paths.
        """
        # Read an image pair given a random integer index
        AB_path = self.AB_paths[index]
        AB = Image.open(AB_path).convert('RGB')

        # Split AB image into A and B AFTER HAVING BEEN PROCESSED FROM THE SCRIPT "combine_A_and_B.py"
        w, h = AB.size
        w2 = int(w / 2)
        A = AB.crop((0, 0, w2, h))
        B = AB.crop((w2, 0, w, h))

        # Apply the same transform to both A and B
        A_transform = get_transform(Image.BICUBIC)
        B_transform = get_transform(Image.BICUBIC)

        A = A_transform(A)
        B = B_transform(B)

        # Return the processed images and their paths as a dictionary
        return {'A': A, 'B': B, 'A_paths': AB_path, 'B_paths': AB_path}

    def __len__(self):
        """
        Return the total number of image pairs.
        """
        return len(self.AB_paths)

In [ ]:
class ResnetBlock(nn.Module):

    def __init__(self, dim, padding_type, norm_layer, use_dropout, use_bias):
        super(ResnetBlock, self).__init__()
        
        # Build convolutional block
        self.conv_block = self.build_conv_block(dim, padding_type, norm_layer, use_dropout, use_bias)

    def build_conv_block(self, dim, padding_type, norm_layer, use_dropout, use_bias):
        
        # Initialize empty list to store convolutional layers and padding
        conv_block = []
        
        # Set padding size to 0 initially
        p = 0
        
        # Add reflection padding layer to the convolutional block
        conv_block += [nn.ReflectionPad2d(1)]
        
        # Add convolutional layer, normalization layer, and activation function to the convolutional block
        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=p, bias=use_bias), norm_layer(dim), nn.ReLU(True)]
        
        # If use_dropout flag is set to True, add dropout layer to the convolutional block
        if use_dropout:
            conv_block += [nn.Dropout(0.5)]
        
        # Add reflection padding layer to the convolutional block
        conv_block += [nn.ReflectionPad2d(1)]
        
        # Add convolutional layer and normalization layer to the convolutional block
        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=p, bias=use_bias), norm_layer(dim)]

        # Return convolutional block as a sequential layer
        return nn.Sequential(*conv_block)
    
    def forward(self, x):
        
        # Add input tensor x with the convolutional block output
        out = x + self.conv_block(x)  # add skip connections
        
        # Return output tensor
        return out

In [ ]:
# Define a class named TestModel
class TestModel:

    # Initialize the class and create a generator network instance
    def __init__(self):
        
        # Create a ResnetGenerator instance with the specified hyperparameters
        self.netG = ResnetGenerator(input_nc=3, output_nc=3, ngf=64, norm_layer=nn.BatchNorm2d, use_dropout=True, n_blocks=9, padding_type='reflect')
        
    # Set the input for the network, which in this case is an image
    def set_input(self, input):
        
        # Set the real image tensor to the input image tensor passed in and move it to the CPU
        self.real = input['A'].to(torch.device('cpu'))
        
        # Set the image path to the input image path passed in
        self.image_paths = input['A_path']

    # Forward pass through the generator network
    def forward(self):
  
        # Generate a fake image using the input image
        self.fake = self.netG(self.real)
        
        # Return the generated fake image tensor
        return self.fake

In [ ]:
# Defining dataset object
dataset = AlignedDataset('path/to/test', 'test_run')

# Defining the Dataloader
data_loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)

In [ ]:
data_loader

In [ ]:
# Initialization of Model

# gpu_ids = [-1]
pix2pix = TestModel()

In [ ]:
modelpath = 'model_file.pth'
# model_details = ResnetGenerator()
model_details = torch.load(modelpath, map_location=torch.device("cuda" if torch.cuda.is_available() else "cpu"))
# model = model_details['model']
# model.load_state_dict(model_details['state_dict_model'])

In [ ]:
pix2pix.netG.eval()

In [ ]:
# Iterating over each batch of data
for i, data in enumerate(data_loader):
    
    # Setting the input of the pix2pix object with the current batch of data
    pix2pix.set_input(data) 
    
    # Making a forward pass through the pix2pix object
    with torch.no_grad():
        pred = pix2pix.forward()

In [ ]:
# Get the predicted image tensor from the model output
image_tensor = pred.data

# Check the shape of the tensor
image_tensor.shape

# Get the first image tensor from the batch and convert it to a numpy array
image_numpy = image_tensor[0].cpu().float().numpy()

# Tile the numpy array across 3 channels
image_numpy = np.tile(image_numpy, (3, 1, 1))

# Transpose the numpy array to get the channels as the last dimension and scale the pixel values to [0, 255]
image_numpy = (np.transpose(image_numpy, (1, 2, 0)) + 1) / 2.0 * 255.0

# Convert the numpy array to an unsigned integer 8-bit array
image_numpy = image_numpy.astype(np.uint8)

# Save the numpy array as a PNG image
# image_pil = Image.fromarray(image_numpy)
image_numpy.save('result.png')